In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [2]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size)
print(simulation)

target expiry: Mon 11 Mar, 08:00 (63 hours left)
current price: 3911.76
-------------------------------
hit ratio: 41%
cost: 0.038 / value: 0.044, benefit/cost = 1.2x
($) cost: 146.69 / value: 170.54, average gain = 23.85
buy put 3875.0
buy call 3925.0
-------------------------------
hit ratio: 41%
cost: 0.032 / value: 0.038, benefit/cost = 1.2x
($) cost: 125.18 / value: 149.97, average gain = 24.80
buy put 3850.0
buy call 3950.0
-------------------------------
hit ratio: 39%
cost: 0.028 / value: 0.034, benefit/cost = 1.2x
($) cost: 107.57 / value: 131.91, average gain = 24.34
buy put 3825.0
buy call 3975.0
-------------------------------
hit ratio: 38%
cost: 0.023 / value: 0.030, benefit/cost = 1.3x
($) cost: 89.97 / value: 116.06, average gain = 26.09
buy put 3800.0
buy call 4000.0


In [61]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
3675.0,165.970299,NaN,0.0385,0.0445,36.721278,0.009715,0.0115,0.0120
3700.0,147.100489,NaN,0.0340,0.0400,42.851468,0.011337,0.0135,0.0145
3725.0,129.417127,NaN,0.0315,0.0340,50.168105,0.013273,0.0160,0.0170
3750.0,113.098046,NaN,0.0280,0.0305,58.849025,0.015569,0.0190,0.0200
3775.0,98.406021,NaN,0.0255,0.0265,69.157000,0.018297,0.0220,0.0235
3800.0,85.527160,0.022627,0.0225,0.0235,81.278139,NaN,0.0255,0.0270
3825.0,74.329739,0.019665,0.0195,0.0210,95.080717,NaN,0.0285,0.0315
3850.0,64.558905,0.017080,0.0170,0.0180,110.309884,NaN,0.0325,0.0355
3875.0,55.989082,0.014813,0.0150,0.0160,126.740061,NaN,0.0370,0.0405


# Sanity checks

In [16]:
size = 12
option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size=size)
put_weights = [0.] * (2 * size + 1)
call_weights = [0.] * (2 * size + 1)
put_weights[6] = 1.
put_weights[10] = -1.
call_weights[14] = -1.
call_weights[18] = 1.
strategy_value_pct, cost = trading_model.backtest(option_chain_df, put_weights, call_weights)
value = strategy_value_pct.mean()
print(value, cost, value/cost)

/workspaces/ethereum-prices/src/optionspricing.py:221: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'put_value_pct_{count}'] = put_values.divide(df['prices'])


-0.011501863455724307 -0.0107 1.0749405098807763
